# Ansible Galaxy Redfish collections

Version 0.135

## Introduction

The Ansible Galaxy [community.general collection](https://galaxy.ansible.com/community/general) proposes three Redfish modules:
  
  * [Redfish info](https://docs.ansible.com/ansible/latest/collections/community/general/redfish_info_module.html) - Retrieves information like systems and accounts inventories. The exhaustive list of possible commands is in the `CATEGORY_COMMANDS_ALL` array in the [module sources](https://github.com/ansible-collections/community.general/blob/main/plugins/modules/remote_management/redfish/redfish_info.py). 
  * [Redfish command](https://docs.ansible.com/ansible/latest/collections/community/general/redfish_command_module.html) - Performs `set` operations on log management, user management, and power operations (e.g. on, off, reboot, etc.). The exhaustive list of possible commands is in the `CATEGORY_COMMANDS_ALL` array in the [module sources](https://github.com/ansible-collections/community.general/blob/main/plugins/modules/remote_management/redfish/redfish_command.py). 
  * [Redfish config](https://docs.ansible.com/ansible/latest/collections/community/general/redfish_config_module.html) - Performs configuration operations on BIOS and other subsystems. The exhaustive list of possible commands is in the `CATEGORY_COMMANDS_ALL` array in the [module sources](https://github.com/ansible-collections/community.general/blob/main/plugins/modules/remote_management/redfish/redfish_config.py). 
  
The above Ansible Redfish modules are based upon the [redfish_utils.py](https://github.com/ansible-collections/community.general/blob/main/plugins/module_utils/redfish_utils.py) python utility module.


Redfish Ansible Playbook examples using the above modules are present in the [DMTF Redfish-Ansible-Playbooks](https://github.com/DMTF/Redfish-Ansible-Playbooks) public GitHub repository.

> **Note**: The Ansible Galaxy Redfish modules can be extended using the [DMTF](https://github.com/DMTF/Redfish-Ansible-Playbooks/blob/master/OEM_EXTENSIONS.md) Oem extensions instructions. This part is not covered in this workshop.


## <a id = "EnvironmentPreparation">Environment preparation</a>

The following cell sets environment variables and checks the connectivity toward the various BMCs used in this notebook.

In [ ]:
########## Environment preparation (Version: 0.135) ############

# Set Student ID number
export stdid={{ STDID }}
Id=$(id --user --name)
NbId=4
InvFile=${NbId}/hosts

# location and ports variables
IloSyBasePort={{ ILOSYBASEPORT }}
let iLO5SimulatorBasePort=$IloSyBasePort
let iLO5SimulatorPort=${iLO5SimulatorBasePort}+${stdid}

iLO5SimulatorIP=ilo5simulators
iLOSimulator=${iLO5SimulatorIP}:${iLO5SimulatorPort}
iLO5SimulatorURI=https://${iLOSimulator}

# Fake Credentials/Token as we are testing against a BMC simulator
SessionToken="FakeToken"

# Miscellaneous
WorkshopDir=$PWD
HpePythonRedfishVenv="${NbId}/HpePythonRedfishVenv"
export PYTHONPATH="${WorkshopDir}/${NbId}/library/"
w=$(basename $PWD)

alias ResetSimulators="../create-globalbmc.shc.x &>/dev/null ; sleep 1"

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in iLO5Simulator ; do
    ip="${bmc}IP" ; port="${bmc}Port"  #; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &>/dev/null && 
        echo -e "\n\tGood News: $bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done


# Create the Ansible inventory file
cat > ${InvFile} << __EOF__
[OneViewManagedBmcs]
${iLO5SimulatorIP} ansible_port=${iLO5SimulatorPort}

[OneViewManagedBmcs:vars]
ansible_python_interpreter=${WorkshopDir}/${HpePythonRedfishVenv}/bin/python3
ansible_search_path=${HpePythonRedfishVenv}
# Below is a fake session token as we are testing against an iLO 5 simulator
token="${SessionToken}"
__EOF__


# Retrieve iLO firmware versions from ServiceRoot (no credentials needed)
for bmc in iLO5Simulator ; do
    ip="${bmc}IP" ; port="${bmc}Port"
    echo -n -e "\t${bmc} firmware version: "
     curl --silent --insecure -X GET https://$(eval echo \$${ip}):$(eval echo \$${port})/redfish/v1 | \
         jq  '[.Oem.Hpe.Manager[]] | .[] | .ManagerFirmwareVersion'
done

## Virtual Python environment creation

In order to completely isolate this notebook environment from other notebooks and student environments, it is safer to create a dedicated Python and Ansible virtual environment.

> **Note**: The installation of Ansible in this `Venv` will also install the `community.general` Galaxy collection in your `~/.ansible` personal directory.

> NOTE: This `Venv` creation can take up to **2 minutes**. Just wait until message `Finished creating Venv` is displayed.



In [ ]:
# Create Virtual Python environment (Venv)
[ -d ${HpePythonRedfishVenv} ] && rm -rf ${HpePythonRedfishVenv} &>/dev/null 
python3 -m venv ${HpePythonRedfishVenv}                         &>/dev/null
source ${HpePythonRedfishVenv}/bin/activate                     &>/dev/null
PS1="[PEXP\[\]ECT_PROMPT>"                                      # Avoid Venv long prompt messing up outputs

# Install latest Ansible in the Venv
pip install wheel                                               &>/dev/null
pip install jmespath                                            &>/dev/null
pip install ansible                                             &>/dev/null

echo -e "\n\n\tFinished creating Venv\n\n"

### <a id = "iLO5SimulatorRestart">Restart iLO 5 simulator</a>

If you need or desire to restart your iLO 5 simulator to restart this workshop from scratch or for other reasons, run the following cell at any time.

In [ ]:
# iLO 5 Simulator restart
ResetSimulators 

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &> /dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done

## Get and set Redfish properties using Ansible Galaxy collections

This third method of using Ansible for Redfish tasks uses the [redfish_info](https://docs.ansible.com/ansible/latest/collections/community/general/redfish_info_module.html) and [redfish_command](https://docs.ansible.com/ansible/latest/collections/community/general/redfish_command_module.html) Galaxy collections to get and set the Indicator LEDs of an HPE Synergy compute node and its enclosure. Then, it does the same exercise against a rack-mount server.

The Redfish Ansible collections presented here are very handy in RedHat [Tower](https://www.ansible.com/tower) or [AWX](https://github.com/ansible/awx) environments as they are built-in and can be extended using the  [DMTF instructions](https://github.com/DMTF/Redfish-Ansible-Playbooks/blob/master/OEM_EXTENSIONS.md).


### Indicator LED management

The next cell launches an Ansible Playbook using an authentication token for performing the same three sections as the previous playbooks:

  * Get status of indicator LEDs 
  * Modify indicator LED(s) status
  * Verify that the action was successful.


A convenient way to study the playbook of the next cell is to open it in a different view in this pane. Right click on this Notebook tab name and select `New View for Notebook` to open a new view.
<img src="Pictures/4-NewViewForNotebook.png" align=middle alt="New View for Notebook" height=30px />

Then, click on this 
[file link](4/SetIndicatorLEDUsingGalaxy.yml).

If you need more space, type `Ctrl-B` (or `Command-B` on a Mac) to hide the left pane. You can have it reappear by hitting `Ctrl-B` again.

Read carefully all the embedded comments in the `.yml` file of the following cell to better understand how the Ansible Galaxy collection works.

In [ ]:
# Set Indicator LED using Galaxy community.general collections against a Synergy compute node
ansible-playbook -i ${InvFile} ${NbId}/SetIndicatorLEDUsingGalaxySessions.yml

### Test the same playbook against a rack-mount server

The following cell switches your environment toward an **HPE DL360 Gen10** simulator and then runs again the Ansible Playbook. You will notice that the same playbook works for both a Synergy compute node and a rack-mount server although it is not enclosed in any frame or enclosure chassis.

In [ ]:
# location and ports variables
IloDlBasePort={{ ILODLBASEPORT }}
let iLO5SimulatorBasePort=$IloDlBasePort
let iLO5SimulatorPort=${iLO5SimulatorBasePort}+${stdid}

iLO5SimulatorIP=ilo5simulators
iLOSimulator=${iLO5SimulatorIP}:${iLO5SimulatorPort}
iLO5SimulatorURI=https://${iLOSimulator}

# Adapt the Ansible inventory file
cat > ${InvFile} << __EOF__
[OneViewManagedBmcs]
${iLO5SimulatorIP} ansible_port=${iLO5SimulatorPort}

[OneViewManagedBmcs:vars]
ansible_python_interpreter=${WorkshopDir}/${HpePythonRedfishVenv}/bin/python3
ansible_search_path=${HpePythonRedfishVenv}
# Below is a fake session token as we are testing against an iLO 5 simulator
token="${SessionToken}"
__EOF__


# Set Indicator LED using Galaxy community.general collections against a rack-mount DL360 Gen10 server
ansible-playbook -i ${InvFile} ${NbId}/SetIndicatorLEDUsingGalaxySessions.yml

### Test the playbook against a Superdome Flex 

The following cell switches your environment toward a **Superdome Flex RMC** simulator and then runs again the Ansible Playbook.

In [ ]:
# location and ports variables
RmcSdfBasePort=47000
let RmcSimulatorBasePort=$RmcSdfBasePort
let RmcSimulatorPort=${RmcSimulatorBasePort}+${stdid}

RmcSimulatorIP=ilo5simulators
RmcSimulator=${RmcSimulatorIP}:${RmcSimulatorPort}
RmcSimulatorURI=https://${RmcSimulator}

# Adapt the Ansible inventory file
cat > ${InvFile} << __EOF__
[OneViewManagedBmcs]
${RmcSimulatorIP} ansible_port=${RmcSimulatorPort}

[OneViewManagedBmcs:vars]
ansible_python_interpreter=${WorkshopDir}/${HpePythonRedfishVenv}/bin/python3
ansible_search_path=${HpePythonRedfishVenv}
# Below is a fake session token as we are testing against an SDF 280 simulator
token="${SessionToken}"
__EOF__


# Toggle Indicator LED(s) using Galaxy community.general collections against an HPE Superdome Flex 
ansible-playbook -i ${InvFile} ${NbId}/SetIndicatorLEDUsingGalaxySessions.yml

## Summary

In this workshop, you used two modules from the `community.general` Ansible Galaxy collection to modify the `IndicatorLED` of an HPE Synergy compute node and its enclosure. This playbook is generic and works also for rack-mount servers and HPE Superdome Flex. You verified this assertion.

The crawling of the Redfish tree to locate the IndicatorLED resources has been mainly performed by the provided Ansible collections. However, you had to extract the properties using the Ansible `yml` syntax in a similar manner as what you did in the second notebook of this workshop.

The Ansible Redfish collections are constantly growing, in terms of new commands. However, if you don't find what you need in the Ansible Redfish collections, you can extend them with your own code as mentioned by the [DMTF](https://github.com/DMTF/Redfish-Ansible-Playbooks/blob/master/OEM_EXTENSIONS.md).

If you are finished, you can go to the [next Notebook](5-RedfishAnsibleUsingIlorest.ipynb).